<a href="https://colab.research.google.com/github/patrickbryant1/Umol/blob/master/Umol.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Umol** - **U**niversal **mol**ecular framework
**Structure prediction of protein-ligand complexes from sequence information**

The protein is represented with a multiple sequence alignment and the ligand as a SMILES string, allowing for unconstrained flexibility in the protein-ligand interface. At a high accuracy threshold, unseen protein-ligand complexes can be predicted more accurately than for RoseTTAFold-AA, and at medium accuracy even classical docking methods that use known protein structures as input are surpassed.

For local installation, see: https://github.com/patrickbryant1/Umol
\
[Read the paper here](https://www.biorxiv.org/content/10.1101/2023.11.03.565471v1)

Umol has **no size limit**. This only depends on available RAM.

Umol is available under the [Apache License, Version 2.0](http://www.apache.org/licenses/LICENSE-2.0). \
The Umol parameters are made available under the terms of the [CC BY 4.0 license](https://creativecommons.org/licenses/by/4.0/legalcode).


In [ ]:
#@title Install dependencies
#@markdown Make sure your runtime is GPU.
#@markdown In the menu above do: Runtime --> Change runtime type --> Hardware accelerator (set to GPU)

#@markdown **Press play.**

#@markdown Simply press play on each cell below and follow the instructions.

#@markdown You will have to restart the runtime after this finishes to include the new packages.
#@markdown In the menu above do: Runtime --> Restart runtime

#@markdown Don't worry about all the errors that pip give below, these are resolved in the end.
#!pip install -U jaxlib==0.3.24+cuda11.cudnn82 -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
!pip install --upgrade "jax[cuda12_local]" -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
#!pip install jax==0.3.24
!pip install ml-collections==0.1.1
!pip install dm-haiku==0.0.11
!pip install pandas==1.3.5
!pip install biopython==1.81
!pip install chex==0.1.5
!pip install dm-tree==0.1.8
!pip install immutabledict==2.0.0
!pip install numpy==1.21.6
!pip install scipy==1.7.3
!pip install tensorflow==2.11.0
!pip install rdkit-pypi
!pip install py3Dmol

In [ ]:
#@title Clone the Umol github repo
import shutil
try:
  shutil.rmtree('/content/Umol', ignore_errors=True)
except:
  print('')

!git clone https://github.com/patrickbryant1/Umol.git

In [8]:
#@title #Follow all steps outlined below to run Umol
#@markdown To try the **test case** 7NB4, click the box "test_case". Then press the play button to the left.
\
#@markdown If you don't want to run the test case, **leave the box blank**.
\
#@markdown The target positions are visualised on a structure generated from [ESMFold](https://www.science.org/doi/10.1126/science.ade2574).
#@markdown You can opt out of adding target positions, but the accuracy is increased if you do.

#@markdown #Settings
#@markdown - *ID* - name \
#@markdown - **MSA** - currently no MSA search is available directly in the browser, therefore you have to provide your own MSAs in a3m format and upload them here. \
#@markdown - Generating an MSA takes a few minutes: \
#@markdown Go to https://toolkit.tuebingen.mpg.de/tools/hhblits \
#@markdown Paste your protein sequence in the search field in fasta format --> Submit. \
#@markdown When the search is finished, go to the tab "Query MSA" and "Download Full A3M" \
#@markdown Upload the MSAs here: \
#@markdown Click the folder icon (Files) to the left and select the upload file icon. Upload your files.
#@markdown Make sure to name your MSA **"ID".a3m**

#@markdown - LIGAND - smiles string of your ligand. Make sure these are canonical (e.g. as generated by RDKit)
#@markdown - SEQUENCE - protein sequence (same as used for the MSA). **Limit=400 amino acids**.
#@markdown You can crop the structure around the target region if it is too big - Umol will handle it.
#@markdown - TARGET_POS - **OPTIONAL** (leave blank if none). What positions to target in the protein sequence (the binding pocket, starting at 1). Note that these have to be all CBs within 10 Å from the putative ligand position.
#@markdown - NUM_RECYCLES - how many recycles to use in the network (increase for harder targets)


import sys, os
from google.colab import files
import pandas as pd
import numpy as np
import py3Dmol

sys.path.insert(0,'/content/Umol/src')
test_case = True #@param {type:"boolean"}
ID = "7NB4" #@param {type:"string"}
LIGAND = "CCc1sc2ncnc(N[C@H](Cc3ccccc3)C(=O)O)c2c1-c1cccc(Cl)c1C" # @param {type:"string"}
SEQUENCE = "SEDELYRQSLEIISRYLREQATGAKDTKPMGRSGATSRKALETLRRVGDGVQRNHETAFQGMLRKLDIKNEDDVKSLSRVMIHVFSDGVTNWGRIVTLISFGAFVAKHLKTINQESCIEPLAESITDVLVRTKRDWLVKQRGWDGFVEFFH" #@param {type:"string"}
TARGET_POSITIONS = "" #@param {type:"string"}
if len(TARGET_POSITIONS)>0:
  TARGET_POSITIONS = np.array([int(x) for x in TARGET_POSITIONS.split(',')])
else:
  TARGET_POSITIONS = []
NUM_RECYCLES = 3 # @param {type:"integer"}
OUTDIR="/content/"+ID+'/'
if not os.path.exists(OUTDIR):
  os.mkdir(OUTDIR)

 #Write fasta
with open('/content/'+ID+'.fasta', 'w') as file:
  file.write('>'+ID+'\n')
  file.write(SEQUENCE)
FASTA_FILE='/content/'+ID+'.fasta'

#Check MSA
if test_case!=True:
  from check_msa_colab import process_a3m
  MSA='/content/'+ID+'.a3m'
  PROCESSED_MSA=MSA.split('.')[0]+'_processed.a3m'
  process_a3m(MSA, SEQUENCE, PROCESSED_MSA)
  MSA=PROCESSED_MSA
else:
  MSA='/content/Umol/data/test_case/'+ID+'/'+ID+'.a3m'


print('Using ligand:', LIGAND)
#print('Using MSA:' ,MSA)
print('Using',NUM_RECYCLES,'recycles')

#Visualise with ESMFold
if not os.path.exists(OUTDIR+'/'+ID+'_esmfold.pdb'):
  !curl -k -X POST --data $SEQUENCE  https://api.esmatlas.com/foldSequence/v1/pdb/ >> $OUTDIR/$ID'_esmfold.pdb'


view = py3Dmol.view(js='https://3dmol.org/build/3Dmol.js',)
view.addModel(open(OUTDIR+ID+"_esmfold.pdb",'r').read(),'pdb')
view.setStyle({'chain':'A'},{'cartoon': {'color':'green'}})

#Highlight
with open(OUTDIR+ID+"_esmfold.pdb","r") as ifile:
    system = "".join([x for x in ifile])
i = 0
for line in system.split("\n"):
    split = line.split()
    if len(split) == 0 or split[0] != "ATOM":
        continue
    idx = int(split[5])
    if idx in TARGET_POSITIONS:
        color = "green"
        view.setStyle({'model': -1, 'serial': i+1}, {"stick": {'color': color}})
    i += 1

view.zoomTo()
view.show()

print('The target structure is in cartoon and the target positions (pocket) in stick format.')

#Subtract 1 from the target positions to make them zero indexed
if len(TARGET_POSITIONS)>0:
  TARGET_POSITIONS -= 1

Using ligand: CCc1sc2ncnc(N[C@H](Cc3ccccc3)C(=O)O)c2c1-c1cccc(Cl)c1C
Using 3 recycles


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

The target structure is in cartoon and the target positions (pocket) in stick format.


In [ ]:
#@markdown #Get the parameters (if not downloaded)
if not os.path.exists('/content/params40000.npy'):
  print('Getting Umol-pocket network parameters.')
  #!wget https://zenodo.org/records/10048543/files/params40000.npy
  !wget https://zenodo.org/records/10397462/files/params40000.npy

if not os.path.exists('/content/params60000.npy'):
  print('Getting Umol network parameters.')
  !wget https://zenodo.org/records/10489242/files/params60000.npy

#@markdown #Make the input features for the network
from make_msa_seq_feats_colab import process
import pickle
#Make MSA feats
feature_dict = process(FASTA_FILE, [MSA])
#Write out features as a pickled dictionary.
features_output_path = os.path.join(OUTDIR, 'msa_features.pkl')
with open(features_output_path, 'wb') as f:
    pickle.dump(feature_dict, f, protocol=4)
print('Saved MSA features to',features_output_path)


#Ligand feats
from make_ligand_feats_colab import bonds_from_smiles
#Atom encoding - no hydrogens
atom_encoding = {'B':0, 'C':1, 'F':2, 'I':3, 'N':4, 'O':5, 'P':6, 'S':7,'Br':8, 'Cl':9, #Individual encoding
                'As':10, 'Co':10, 'Fe':10, 'Mg':10, 'Pt':10, 'Rh':10, 'Ru':10, 'Se':10, 'Si':10, 'Te':10, 'V':10, 'Zn':10 #Joint (rare)
                 }

#Get the atom types and bonds
atom_types, atoms, bond_types, bond_lengths, bond_mask = bonds_from_smiles(LIGAND, atom_encoding)

ligand_inp_feats = {}
ligand_inp_feats['atoms'] = atoms
ligand_inp_feats['atom_types'] = atom_types
ligand_inp_feats['bond_types'] = bond_types
ligand_inp_feats['bond_lengths'] = bond_lengths
ligand_inp_feats['bond_mask'] = bond_mask
#Write out features as a pickled dictionary.

features_output_path = os.path.join(OUTDIR, 'ligand_inp_features.pkl')
with open(features_output_path, 'wb') as f:
    pickle.dump(ligand_inp_feats, f, protocol=4)
print('Saved ligand features to',features_output_path)

In [ ]:
#@markdown #Predict the protein-ligand structure (a few minutes)
from net.model import config
from predict_colab import predict
import numpy as np
MSA_FEATS='/content/'+ID+'/msa_features.pkl'
LIGAND_FEATS='/content/'+ID+'/ligand_inp_features.pkl'
if len(TARGET_POSITIONS)>0:
  print('Target positions specified. Using pocket version.')
  PARAMS=np.load('/content/params40000.npy', allow_pickle=True)
else:
  print('No target positions specified. Using no-pocket version.')
  PARAMS=np.load('/content/params60000.npy', allow_pickle=True)

#Predict
predict(config.CONFIG,
            MSA_FEATS,
            LIGAND_FEATS,
            ID,
            TARGET_POSITIONS,
            PARAMS,
            NUM_RECYCLES,
            outdir=OUTDIR)
from relax.align_ligand_conformer_colab import read_pdb, generate_best_conformer, align_coords_transform, write_sdf
import pandas as pd
RAW_PDB=OUTDIR+'/'+ID+'_pred_raw.pdb'
#Get a conformer
pred_ligand = read_pdb(RAW_PDB)
best_conf, best_conf_pos, best_conf_err, atoms, nonH_inds, mol, best_conf_id  = generate_best_conformer(pred_ligand['chain_coords'], LIGAND)

#Align it to the prediction
aligned_conf_pos = align_coords_transform(pred_ligand['chain_coords'], best_conf_pos, nonH_inds)

#Write sdf
write_sdf(mol, best_conf, aligned_conf_pos, best_conf_id, OUTDIR+ID+'_pred_ligand.sdf')
!grep ATOM $RAW_PDB > $OUTDIR/$ID'_pred_protein.pdb'

#Ligand plDDT
!grep HETATM $RAW_PDB| cut -c65-66 > $OUTDIR/ligand_plddt.csv

In [6]:
#@markdown #Visualise
import py3Dmol
import pandas as pd
import numpy as np

ligand_plddt = pd.read_csv(OUTDIR+'ligand_plddt.csv', header=None)
print('The average ligand plDDT is:', np.round(ligand_plddt[0].mean(),1))

view = py3Dmol.view(js='https://3dmol.org/build/3Dmol.js',)
view.addModel(open(OUTDIR+ID+"_pred_ligand.sdf",'r').read(),'sdf')
view.setStyle({'stick': {'color':'cyan'}})
view.addModel(open(OUTDIR+ID+"_pred_protein.pdb",'r').read(),'pdb')
view.setStyle({'chain':'A'},{'cartoon': {'color':'green'}})
view.zoomTo()
view.show()

The average ligand plDDT is: 80.6


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [ ]:
#@title Download unrelaxed results
from google.colab import files
files.download(OUTDIR+ID+"_pred_ligand.sdf")
files.download(OUTDIR+ID+"_pred_protein.pdb")

In [ ]:
#@markdown # **Relaxation** - this section requires additional installations *but* it can help fix clashes in the predicted protein

#@markdown ###Press play and install conda for relaxation with OpenMM
!pip install -q condacolab
import condacolab
condacolab.install()

In [ ]:
#@markdown ###Install OpenMM in a conda environment (this can take a while).
%%bash
conda create -c conda-forge --name openmm openmm
source activate openmm
conda install -c conda-forge openff-toolkit
conda install -c conda-forge pdbfixer
conda install -c omnia openmmforcefields


In [ ]:
#@markdown ###**Relax the structure**
#@markdown ### You have to double click on this cell and change the ID if you are not running the test case.

%%bash
source activate openmm

python
ID='7NB4' #Type the ID in here!
import openmm as mm
import argparse
import sys
import numpy as np
import openmm.app as mm_app
import openmm.unit as mm_unit
import pdbfixer
import pdb
from sys import stdout
from openmm.app import PDBFile, Modeller
import mdtraj
from openmmforcefields.generators import SystemGenerator
from openff.toolkit import Molecule
from openff.toolkit.utils.exceptions import UndefinedStereochemistryError, RadicalsNotSupportedError
from openmm import CustomExternalForce

###########Functions##########
def fix_pdb(
        pdbname,
        outdir,
        file_name
        ):
    """Add hydrogens to the PDB file
    """
    fixer = pdbfixer.PDBFixer(pdbname)
    fixer.findMissingResidues()
    fixer.findNonstandardResidues()
    fixer.replaceNonstandardResidues()
    fixer.findMissingAtoms()
    fixer.addMissingAtoms()
    fixer.addMissingHydrogens(7.0)
    mm_app.PDBFile.writeFile(fixer.topology, fixer.positions, open(f'{outdir}/{file_name}_hydrogen_added.pdb', 'w'))
    return fixer.topology, fixer.positions

def set_system(topology):
    """
    Set the system using the topology from the pdb file
    """
    #Put it in a force field to skip adding all particles manually
    forcefield = mm_app.ForceField('amber14-all.xml', 'amber14/tip3pfb.xml')
    system = forcefield.createSystem(topology,
                                     removeCMMotion=False,
                                     nonbondedMethod=mm_app.NoCutoff,
                                     rigidWater=True #Use implicit solvent
                                     )
    return system

def minimize_energy(
        topology,
        system,
        positions,
        outdir,
        out_title
        ):
    '''Function that minimizes energy, given topology, OpenMM system, and positions '''
    #Use a Brownian Integrator
    integrator = mm.BrownianIntegrator(
        100 * mm.unit.kelvin,
        100. / mm.unit.picoseconds,
        2.0 * mm.unit.femtoseconds
    )
    simulation = mm.app.Simulation(topology, system, integrator)

    # Initialize the DCDReporter
    reportInterval = 100  # Adjust this value as needed
    reporter = mdtraj.reporters.DCDReporter('positions.dcd', reportInterval)

    # Add the reporter to the simulation
    simulation.reporters.append(reporter)

    simulation.context.setPositions(positions)

    simulation.minimizeEnergy(1, 1000)
    # Save positions
    minpositions = simulation.context.getState(getPositions=True).getPositions()
    mm_app.PDBFile.writeFile(topology, minpositions, open(outdir+f'{out_title}.pdb','w'))

    reporter.close()

    return topology, minpositions

def add_restraints(
        system,
        topology,
        positions,
        restraint_type
        ):
    '''Function to add restraints to specified group of atoms

    Code adapted from https://gist.github.com/peastman/ad8cda653242d731d75e18c836b2a3a5

    '''
    restraint = CustomExternalForce('k*periodicdistance(x, y, z, x0, y0, z0)^2')
    system.addForce(restraint)
    restraint.addGlobalParameter('k', 100.0*mm_unit.kilojoules_per_mole/mm_unit.nanometer**2)
    restraint.addPerParticleParameter('x0')
    restraint.addPerParticleParameter('y0')
    restraint.addPerParticleParameter('z0')

    for atom in topology.atoms():
        if restraint_type == 'protein':
            if 'x' not in atom.name:
                restraint.addParticle(atom.index, positions[atom.index])
        elif restraint_type == 'CA+ligand':
            if ('x' in atom.name) or (atom.name == "CA"):
                restraint.addParticle(atom.index, positions[atom.index])

    return system
## Read in ligand
print('Reading ligand')

ligand_mol = Molecule.from_file("/content/"+ID+"/"+ID+"_pred_ligand.sdf")

# Assigning partial charges first because the default method (am1bcc) does not work
ligand_mol.assign_partial_charges(partial_charge_method='gasteiger')

## Read protein PDB and add hydrogens
protein_topology, protein_positions = fix_pdb("/content/"+ID+"/"+ID+"_pred_protein.pdb", "/content/"+ID+"/", ID)
print('Added all atoms...')

# Minimize energy for the protein
system = set_system(protein_topology)
print('Creating system...')
#Relax
print('Preparing complex')
## Add protein first
modeller = Modeller(protein_topology, protein_positions)
print('System has %d atoms' % modeller.topology.getNumAtoms())

## Then add ligand
print('Adding ligand...')
lig_top = ligand_mol.to_topology()
modeller.add(lig_top.to_openmm(), lig_top.get_positions().to_openmm())
print('System has %d atoms' % modeller.topology.getNumAtoms())

print('Preparing system')
# Initialize a SystemGenerator using the GAFF for the ligand and implicit water.
# forcefield_kwargs = {'constraints': mm_app.HBonds, 'rigidWater': True, 'removeCMMotion': False, 'hydrogenMass': 4*mm_unit.amu }
system_generator = SystemGenerator(
    forcefields=['amber14-all.xml', 'implicit/gbn2.xml'],
    small_molecule_forcefield='gaff-2.11',
    molecules=[ligand_mol],
    # forcefield_kwargs=forcefield_kwargs
)

## Create system
system = system_generator.create_system(modeller.topology, molecules=ligand_mol)

print('Adding restraints on protein CAs and ligand atoms')

system = add_restraints(system, modeller.topology, modeller.positions, restraint_type='CA+ligand')

## Minimize energy
minimize_energy(
    modeller.topology,
    system,
    modeller.positions,
    "/content/"+ID+"/",
    ID+'_relaxed_complex'
)




In [ ]:
#@title Download relaxed results
from google.colab import files
import glob
for relaxed_complex in glob.glob('/content/*/*_relaxed_complex.pdb'):
  files.download(relaxed_complex)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>